In [4]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 25 20:32:30 2021

@author: Matt Pitz
"""


import user_data as user
import database_functions as dbf
import recommender as rec

def main():
    #collect user inputs (note only works on new user for now
    new_user, user_id, gender, age_group, month, updated_user_df = user.get_user_data()
    
    #collect user goals
    weekly_target, days_to_run, medium_intensity_runs, high_intensity_runs, long_run = user.get_run_plan()
    
    #load db in memory (Move this to server start when website work)
    raw_df = dbf.load_data()
    print('finished loading')
    
    #pass user data to db
    update_user_df = dbf.update_database(new_user, gender, age_group, month, weekly_target, days_to_run, user_id, updated_user_df)
    
    #create dataset with user data for model
    filtered_df = dbf.database_for_recommender(raw_df, update_user_df, gender, age_group, month, days_to_run, weekly_target)
    
    #return recommendations
    recommendations_df = rec.generate_run_ratings(filtered_df,user_id, weekly_target, days_to_run)
    
    #return training plan
    run_schedule = rec.return_run_schedule(recommendations_df, days_to_run, weekly_target, medium_intensity_runs, high_intensity_runs, long_run)
    
    return run_schedule

schedule = main()
schedule
#eof

Are you a new user? (yes/no):  no
Please enter your user id:  14839
How many days did you run in the last 7 days?:  1
What is your gender? (M/F/Other)  M
What is your age?  35
Enter a date you ran in MM/DD/YYYY format:  08/01/2023
How many kilometers did you run?:  20
What was the total run time in HH:MM:SS format?:  00:56:00
How many km do you plan to run this week?  60
How many days do you plan to run?  6
How many medium intensity runs would you like?  1
How many high intensity runs would you like?  1
Would you like a Sunday long run? (yes/no)  yes


finished loading


/home/ec2-user/Capstone/App/database_functions.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  user_df['Duration'] = pd.to_datetime(user_df['Duration'])
/home/ec2-user/Capstone/App/database_functions.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_month_runs['run_distance_rounded'] = curr_month_runs.Distance.round()


,run_distance,pace
4,9,00:06:30
0,10,00:06:30
5,10,00:06:30
1,10,00:06:30
2,8,00:06:30
3,13,00:06:30


In [3]:
#Snippet to remove user data when testing
"""
import pandas as pd
from datetime import datetime
import pyarrow.parquet as pq

user_df = pd.read_parquet('../App_Data/new_user_db.parquet')
date_str = '2023-08-01'
format_str = '%Y-%m-%d'
datetime_obj = datetime.strptime(date_str, format_str).date()
user_df = user_df[user_df['Date']<datetime_obj]
user_df
user_df.to_parquet('../App_Data/new_user_db.parquet')
"""

"\nimport pandas as pd\nfrom datetime import datetime\nimport pyarrow.parquet as pq\n\nuser_df = pd.read_parquet('../App_Data/new_user_db.parquet')\ndate_str = '2023-08-01'\nformat_str = '%Y-%m-%d'\ndatetime_obj = datetime.strptime(date_str, format_str).date()\nuser_df = user_df[user_df['Date']<datetime_obj]\nuser_df\nuser_df.to_parquet('../App_Data/new_user_db.parquet')\n"